# Interactive maps

In this tutorial we will learn how to publish data from Python on interactive [leaflet.js](http://leafletjs.com/) maps. 

JavaScript (JS)  is a programming language for adding interactive content (such a zoomamble maps!) on webpages. [Leaflet](http://leafletjs.com/) is a popular JavaScript library for creating interactive maps for webpages ([OpenLayers](https://openlayers.org/) is another JavaScript library for the same purpose). 

Here, will mainly focus on [Folium](https://python-visualization.github.io/folium/) - a Python library that makes it easy to convert data from (Geo)DataFrames into interactive Leaflet maps.

<div class="alert alert-info">

**Explore also...**
    
Other interesting libraries for creating interactive visualizations from spatial data:
    
- [mapboxgl](https://github.com/mapbox/mapboxgl-jupyter)
- [Bokeh](https://docs.bokeh.org/en/latest/)
- [Geoviews](http://geoviews.org/)
- [plotly express](https://plotly.com/python/maps/)

</div>


## Folium


[Folium](https://github.com/python-visualization/folium)  is a Python library that makes
it possible visualize data on an interactive Leaflet map.

**Resources:**

- [Folium Documentation](https://python-visualization.github.io/folium/)
- [Example Gallery](https://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/)
- [Folium Quickstart](https://python-visualization.github.io/folium/quickstart.html)


### Creating a simple interactive web-map

Import folium and other useful packages:

In [ ]:
import folium
from folium.plugins import Fullscreen


In [ ]:
from pyproj import crs
import geopandas as gpd
import matplotlib.pyplot as plt

We will start by creating a simple interactive web-map without any data on it. We just visualize OpenStreetMap on a specific location of the world.

First thing that we need to do is to create [a Map instance](https://python-visualization.github.io/folium/modules.html#folium.folium.Map) and define a location for zooming in the data: 

In [ ]:
# Create a Map instance
m = folium.Map(location=[60.25, 24.8], zoom_start=10, control_scale=True)

The first parameter ``location`` takes a pair of lat, lon values as list as an input which will determine where the map will be positioned when user opens up the map. ``zoom_start`` -parameter adjusts the default zoom-level for the map (the higher the number the closer the zoom is). ``control_scale`` defines if map should have a scalebar or not.

Let's see what our map looks like: 

In [ ]:
m

We can also save the map as a html file:

In [ ]:
outfp = "base_map.html"
m.save(outfp)


You should now see a html file in your working directory. You can open the file in a web-browser in order to see the map, or in a text editor in order to see the source definition.


Let's create another map with different settings (location, bacground map, zoom levels etc). See documentation of the [Map() object](https://python-visualization.github.io/folium/modules.html#folium.folium.Map) for all avaiable options.
    
``tiles`` -parameter is used for changing the background map provider and map style (see the [documentation](https://python-visualization.github.io/folium/modules.html#folium.folium.Map) for all in-built options).


In [ ]:
# Let's change the basemap style to 'Stamen Toner'
m = folium.Map(location=[40.730610, -73.935242], tiles='Stamen Toner',
                zoom_start=12, control_scale=True, prefer_canvas=True)

m

### Adding layers to the map

Let's first have a look how we can add a simple [marker](https://python-visualization.github.io/folium/modules.html?highlight=marker#folium.map.Marker) on the webmap:

In [ ]:
# Create a Map instance
m = folium.Map(location=[60.20, 24.96],
    zoom_start=12, control_scale=True)

# Add marker
# Run: help(folium.Icon) for more info about icons
folium.Marker(
    location=[60.20426, 24.96179],
    popup='Kumpula Campus',
    icon=folium.Icon(color='green', icon='ok-sign'),
).add_to(m)

#Show map
m

As mentioned, Folium combines the strenghts of data manipulation in Python with the mapping capabilities of Leaflet.js. Eventually, we would like to include the plotting of interactive maps as the last part of our data analysis workflow. 

Let's see how we can plot data from a geodataframe using folium.



In [ ]:
# File path
points_fp = r"data/addresses.shp"

# Read the data
points = gpd.read_file(points_fp)

#Check input data
points.head()

In [ ]:
points.head()

- conver the points to GeoJSON features using folium:

In [ ]:
# Convert points to GeoJSON
points_gjson = folium.features.GeoJson(points, name="Public transport stations")

In [ ]:
# Check the GeoJSON features
#points_gjson.data.get('features')

Now we have our population data stored as GeoJSON format which basically contains the
data as text in a similar way that it would be written in the ``.geojson`` -file.

Add the points onto the Helsinki basemap:

In [ ]:
# Create a Map instance
m = folium.Map(location=[60.25, 24.8], tiles = 'cartodbpositron', zoom_start=11, control_scale=True)

# Add points to the map instance
points_gjson.add_to(m)

# Alternative syntax for adding points to the map instance
#m.add_child(points_gjson)

#Show map
m

### Layer control

We can also add a `LayerControl` object on our map, which allows the user to control which map layers are visible. See the [documentation](http://python-visualization.github.io/folium/docs-v0.5.0/modules.html#folium.map.LayerControl) for available parameters (you can e.g. change the position of the layer control icon).

In [ ]:
# Create a layer control object and add it to our map instance
folium.LayerControl().add_to(m)

#Show map
m

### Heatmap

[Folium plugins](https://python-visualization.github.io/folium/plugins.html) allow us to use popular tools available in leaflet. One of these plugins is [HeatMap](https://python-visualization.github.io/folium/plugins.html#folium.plugins.HeatMap), which creates a heatmap layer from input points. 

Let's visualize a heatmap of the public transport stations in Helsinki using the addresses input data. [folium.plugins.HeatMap](https://python-visualization.github.io/folium/plugins.html#folium.plugins.HeatMap) requires a list of points, or a numpy array as input, so we need to first manipulate the data a bit:

In [ ]:
# Get x and y coordinates for each point
points["x"] = points["geometry"].x
points["y"] = points["geometry"].y

# Create a list of coordinate pairs
locations = list(zip(points["y"], points["x"]))

Check the data:

In [ ]:
locations

In [ ]:
from folium.plugins import HeatMap

# Create a Map instance
m = folium.Map(location=[60.25, 24.8], tiles = 'stamentoner', zoom_start=10, control_scale=True)

# Add heatmap to map instance
# Available parameters: HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True)
HeatMap(locations).add_to(m)

# Show map
m

### Clustered point map

Let's visualize the address points (locations of transport stations in Helsinki) on top of the choropleth map using clustered markers using folium's [MarkerCluster](https://python-visualization.github.io/folium/plugins.html?highlight=marker%20cluster#folium.plugins.MarkerCluster) class.

In [ ]:
from folium.plugins import MarkerCluster

In [ ]:
# Create a Map instance
m = folium.Map(location=[60.25, 24.8], tiles = 'cartodbpositron', zoom_start=11, control_scale=True)

In [ ]:
# Following this example: https://github.com/python-visualization/folium/blob/master/examples/MarkerCluster.ipynb

# Get x and y coordinates for each point
points["x"] = points["geometry"].x
points["y"] = points["geometry"].y

# Create a list of coordinate pairs
locations = list(zip(points["y"], points["x"]))

In [ ]:
# Create a folium marker cluster
marker_cluster = MarkerCluster(locations)

# Add marker cluster to map
marker_cluster.add_to(m)

# Show map
m

### Choropleth map

Next, let's check how we can overlay polygons to a folium map using LA County's Social Vulnerability Index (https://www.arcgis.com/home/item.html?id=bb8b6e2136384c38b05544235bd67cb8). 

In [ ]:
# Set where you data is located and the file you wish to open
file_path = 'data/Social_Vulnerability_Index_LA_County.geojson'
# Load the GeoJSON again as a GeoDataFrame
gdf = gpd.read_file(file_path)


Examine the data frame...

# Select the column for the choropleth (Poverty Percentage - EP_POV)
choropleth_field = "EP_POV"

Create and interactive map

In [ ]:

# Compute the centroid of the county for the initial map view
center_lat, center_lon = gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()

# Create a Folium map centered on LA County
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles="cartodb positron")

# Add a Choropleth layer
choropleth = folium.Choropleth(
    geo_data=file_path,
    name="Choropleth",
    data=gdf,
    columns=["FIPS", choropleth_field],  # Use FIPS as the unique key
    key_on="feature.properties.FIPS",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Poverty Percentage (%)",
).add_to(m)

### Tooltips

It is possible to add different kinds of pop-up messages and tooltips to the map. Here, it would be nice to see some additional metadata for the individual polygons.  

Add tooltips, we can add tooltips to our map when plotting the polygons as GeoJson objects using the `GeoJsonTooltip` feature. (following examples from [here](http://nbviewer.jupyter.org/gist/jtbaker/57a37a14b90feeab7c67a687c398142c?flush_cache=true) and [here](https://nbviewer.jupyter.org/github/jtbaker/folium/blob/geojsonmarker/examples/GeoJsonMarkersandTooltips.ipynb))

For a quick workaround, we plot the polygons on top of the coropleth map as a transparent layer, and add the tooltip to these objects. *Note: this is not an optimal solution as now the polygon geometry get's stored twice in the output!*

In [ ]:
# Add a tooltip to show exact values on hover
tooltip = folium.GeoJsonTooltip(
    fields=["LOCATION", choropleth_field],
    aliases=["Location:", "Poverty Rate (%):"],
    localize=True,
    sticky=True
)

# Add GeoJson with tooltips
folium.GeoJson(
    file_path,
    name="Poverty Rate",
    tooltip=tooltip,
    style_function=lambda x: {"fillOpacity": 0, "color": "black", "weight": 0.3}
).add_to(m)

# Add fullscreen button
Fullscreen().add_to(m)

m

Rember that you can also save the output as an html file: 

In [ ]:
# Save and display the map
map_path = "LA_County_Poverty_Map.html"
m.save(map_path)

Extra: check out plotly express for an alternative way of plotting an interactive Choropleth map [in here](https://plotly.com/python/mapbox-county-choropleth/).

In [ ]:
import plotly.express as px

# Load GeoJSON
gdf = gpd.read_file("data/Social_Vulnerability_Index_LA_County.geojson")

fig = px.choropleth(
    gdf,
    geojson=gdf.geometry,
    locations=gdf.index,
    color="EP_POV",
    color_continuous_scale="OrRd",
    title="Poverty Rate (%)"
)

fig.update_geos(fitbounds="locations", visible=False)
fig.show()